<a href="https://colab.research.google.com/github/moaaz12-web/Question-Answering-inference-using-BERT/blob/main/Text_summarization_from_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT DEPENDENCIES

In [ ]:
! pip install transformers gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 36.0 MB/s 
     |████████████████████████████████| 5.3 MB 52.1 MB/s 
     |████████████████████████████████| 163 kB 75.4 MB/s 
     |████████████████████████████████| 6.6 MB 26.6 MB/s 
     |████████████████████████████████| 57 kB 5.9 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 112 kB 58.8 MB/s 
     |████████████████████████████████| 270 kB 74.9 MB/s 
     |████████████████████████████████| 2.3 MB 57.8 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 212 kB 74.8 MB/s 
     |████████████████████████████████| 63 kB 2.8 MB/s 
     |████████████████████████████████| 80 kB 9.4 MB/s 
     |████████████████████████████████| 68 kB

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import gradio as gr
import requests

# DOWNLOAD THE MODEL

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

# GET THE URL AND THE DATA

In [ ]:
URL = 'https://www.verywellmind.com/how-to-be-successful-in-life-4165743'
r = requests.get(URL)


In [ ]:
r.text

'<!DOCTYPE html>\n<html id="listScTemplate_1-0" class="comp no-js taxlevel-2 vertical-verywellmind listScTemplate html mntl-html" data-ab="99,99,55,53,63,99,99" data-resource-version="4.318.0" lang="en" data-mantle-resource-version="3.13.478" data-health-resource-version="4.318.0" data-health-mind-resource-version="4.318.0" data-tracking-container="true">\t\t\t\t\n<!--\n<globe-environment environment="k8s-prod" application="health-mind" dataCenter="us-west-1"/>\n-->\n<head class="loc head">\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t<script type="text/javascript">var Mntl = window.Mntl || {};</script>\n\t\t\t\t\t\n\t\t\t\t\t    <link rel="preconnect" href="//js-sec.indexww.com">\n    <link rel="preconnect" href="//c.amazon-adsystem.com">\n    <link rel="preconnect" href="//securepubads.g.doubleclick.net">\n    <link rel="preconnect" href="//ak.sail-horizon.com">\n\n\t\t\t\t\t\n\t\t\t\t\t\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n<meta name="robots" content="

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
res = soup.find_all(['h1', 'p'])

In [ ]:
res

In [ ]:
text = [ress.text for ress in res]
ARTICLE = ''.join(text)

# DATA CLEANING AND PREPROCESSING

In [ ]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [ ]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
max_chunk = 500
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)


In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])


In [ ]:
text

" There are many different tactics for how to be successful in life, but the strategy that works best for you may depend on what success means to you . Your definition of success may vary, but many might define it as being fulfilled, happy, safe, healthy, and loved . People who believe that they are capable of growth are more likely to achieve success .  This episode of The Verywell Mind Podcast shares an exercise that can help you introduce a healthy habit into your life or get rid of a bad habit that's been holding you back . Emotional intelligence refers to the ability to understand, utilize, and reason with emotions .  Researchers Ian MacRae and Adrian Furnham have identified six key traits that can play a role in how well people do at work . Too little of these characteristics can hinder success, but so can having too much . No specific personality trait can guarantee success, nor can being low in that trait doom someone to failure .  There is no single measure of success, and cer

## DEPLOYMENT USING GRADIO

In [ ]:


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize(URL):
  # summarizer = pipeline("summarization")
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  res = soup.find_all(['h1', 'p'])
  text = [ress.text for ress in res]
  ARTICLE = ''.join(text)
  ARTICLE = ARTICLE.replace('.', '.<eos>')
  ARTICLE = ARTICLE.replace('?', '?<eos>')
  ARTICLE = ARTICLE.replace('!', '!<eos>')

  sentences = ARTICLE.split('<eos>')
  current_chunk = 0 
  max_chunk = 500
  chunks = []
  for sentence in sentences:
      if len(chunks) == current_chunk + 1: 
          if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
              chunks[current_chunk].extend(sentence.split(' '))
          else:
              current_chunk += 1
              chunks.append(sentence.split(' '))
      else:
          print(current_chunk)
          chunks.append(sentence.split(' '))

  for chunk_id in range(len(chunks)):
      chunks[chunk_id] = ' '.join(chunks[chunk_id])

  res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)
  text = ' '.join([summ['summary_text'] for summ in res])
  return text



iface = gr.Interface(
  fn=summarize, 
  inputs='text',
  outputs='text',
  
)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://13884.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fee200c6050>,
 'http://127.0.0.1:7860/',
 'https://13884.gradio.app')